<a href="https://colab.research.google.com/github/fahmi143/Karst-2.0/blob/main/Karst2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import imageio as iio
from PIL import Image
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, Conv2D, MaxPooling2D

In [30]:
class_names= ['Dendritic','Sinkhole']
image_size=(150,150)
batch_size=35



In [134]:
train_ds=train.flow_from_directory('/content/drive/MyDrive/KarstImages/TRAIN',target_size=(200,200),
                                        batch_size=3,
                                        class_mode='binary')
val_ds=train.flow_from_directory('/content/drive/MyDrive/KarstImages/VALIDATION',target_size=(200,200),
                                        batch_size=3,
                                        class_mode='binary')
test_ds=train.flow_from_directory('/content/drive/MyDrive/KarstImages/TEST',target_size=(200,200),
                                        batch_size=3,
                                        class_mode='binary')

Found 16 images belonging to 2 classes.
Found 6 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [137]:
#Data Augmentation
training_ds = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True)
validation_ds = ImageDataGenerator(rescale=1./255)
test_ds = ImageDataGenerator(rescale=1./255)

In [139]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
# make 32 feature detectors with a size of 3x3
# choose the input-image's format to be 150x150 with 3 channels
classifier.add(Conv2D(16, (3, 3), input_shape=(150,150, 3), activation="relu"))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a third  convolutional layer
classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(activation="relu", units=128))
classifier.add(Dense(activation="sigmoid", units=1))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# finally start computation
# to improve the model accuracy you can increase the number of steps_per_epoch to e.g. 8000
# increase the number of epochs to 5-25
# increase the validation steps
# this parametters allow for the model to optimize

model_fit= classifier.fit(train_ds,
                     steps_per_epoch=3,
                    epochs=25,
                    validation_data=validation_ds)



Epoch 1/25


InvalidArgumentError: ignored

In [136]:
history = model.fit(
            train_ds,
            validation_data = val_ds,
            epochs = 30,
            verbose = 2)

Epoch 1/30


InvalidArgumentError: ignored

In [74]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 74, 74, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 34, 34, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 17, 17, 64)      

In [116]:
test_loss, test_acc = model.evaluate(test_ds, verbose=2)

print('\nTest accuracy:', test_acc)

3/3 - 1s - loss: 0.6956 - accuracy: 0.6667 - 1s/epoch - 363ms/step

Test accuracy: 0.6666666865348816


In [117]:
preds = model.predict(test_ds)
#if using softmax activation on output layer
predicted_labels  = np.argmax(preds,axis=1)
#Accessing classnames with predicted clases
final_predictions = [class_names[i] for i in predicted_labels]

3/3 [==============================] - 0s 47ms/step
